In [ ]:
depts_tmp = train_grouped.loc[:, "1-HR PHOTO":"WIRELESS"]
depts_tmp = depts_smoothed
depts_tmp.columns = depts_tmp.columns.map(lambda x: str("Prob_") + x)

In [1]:
train = pd.read_csv("../dataset/0408_noFineline_train.csv")
test = pd.read_csv("../dataset/0408_noFineline_test.csv")

In [2]:
train.head()

,VisitNumber,ScanCount,Count,Min,Max,Mean,Range,Null_Depts,ReturnCount,Null_Fineline,...,Prob_TOYS,Prob_WIRELESS,FinelineNumber,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6
0,5,-1,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.014706,0.014706,1000.0,0,0,0,0,1,0,0
1,7,1,1,1.0,1.0,1.000000,0.0,0.0,0.0,0.0,...,0.014286,0.014286,8931.0,0,0,0,0,1,0,0
2,8,3,3,1.0,20.0,4.833333,19.0,0.0,0.0,0.0,...,0.010204,0.010204,3565.0,0,0,0,0,1,0,0
3,9,1,1,1.0,2.0,1.500000,1.0,0.0,0.0,0.0,...,0.014085,0.014085,115.0,0,0,0,0,1,0,0
4,10,1,1,1.0,2.0,1.500000,1.0,0.0,0.0,0.0,...,0.014085,0.014085,2008.0,0,0,0,0,1,0,0


In [5]:
prob_cols = []
cols = train.columns.tolist()
for col in cols:
    if col.startswith("Prob_"):
        prob_cols.append(col)

In [8]:
prob_cols[0], prob_cols[-1]

('Prob_1-HR PHOTO', 'Prob_WIRELESS')

In [13]:
train_wihouth_prob = train.drop(train.loc[:, prob_cols[0]:prob_cols[-1]], axis=1)

In [14]:
test_with_prob = test.drop(test.loc[:, prob_cols[0]:prob_cols[-1]], axis=1)

In [20]:
y_df = pd.read_csv("../dataset/0409_train.csv").set_index("VisitNumber")["TripType"]

In [16]:
train_array = train_wihouth_prob.values
test_array = test_with_prob.values

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train_array, y_df, random_state=0)

In [23]:
rf = RandomForestClassifier(n_estimators=1000, min_samples_split=30, n_jobs=-1, random_state=0)

In [24]:
%%time
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=30,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [25]:
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.736394676329
0.657468957732


In [28]:
pred_prob_test = rf.predict_proba(test_array)
proba_df = pd.DataFrame(pred_prob_test, columns=rf.classes_)
proba_df.columns = proba_df.columns.map(lambda x: "TripType_" + str(x))
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)
sub_df.to_csv("../submission_0409_noProb.csv", index=False)

In [44]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder().fit(y_df.values)
y_labeled = label.transform(y_df.values)

In [53]:
y_labeled_train, y_labeled_test = train_test_split(y_labeled, random_state=0)

In [59]:
import xgboost as xgb
from sklearn.metrics import log_loss

dtrain = xgb.DMatrix(X_train, label=y_labeled_train)
dtest = xgb.DMatrix(X_test, label=y_labeled_test)

In [77]:
num_round = 300
param1 = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss', 
          'num_class': 38,
          'max_depth' : 3,
          "max_delta_step": 3, 
          "eta": 0.25}

watchlist1 = [(dtrain,'train'), (dtest, 'eval')]

bst = xgb.train(param1, dtrain, num_round, watchlist1, early_stopping_rounds=10)

[0]	train-mlogloss:3.00377	eval-mlogloss:3.00471
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.50616	eval-mlogloss:2.50957
[2]	train-mlogloss:2.15493	eval-mlogloss:2.15923
[3]	train-mlogloss:1.94009	eval-mlogloss:1.9461
[4]	train-mlogloss:1.78965	eval-mlogloss:1.79686
[5]	train-mlogloss:1.67662	eval-mlogloss:1.68517
[6]	train-mlogloss:1.58791	eval-mlogloss:1.59692
[7]	train-mlogloss:1.51547	eval-mlogloss:1.52523
[8]	train-mlogloss:1.45498	eval-mlogloss:1.46558
[9]	train-mlogloss:1.40392	eval-mlogloss:1.41584
[10]	train-mlogloss:1.36247	eval-mlogloss:1.37522
[11]	train-mlogloss:1.32655	eval-mlogloss:1.33958
[12]	train-mlogloss:1.29391	eval-mlogloss:1.30771
[13]	train-mlogloss:1.2668	eval-mlogloss:1.28157
[14]	train-mlogloss:1.24353	eval-mlogloss:1.25926
[15]	train-mlogloss:1.2222	eval-mlogloss:1.23874
[16]	train-mlogloss:1.20299	eval-mlogloss:1.22037
[17]	train-m

[157]	train-mlogloss:0.788117	eval-mlogloss:0.936159
[158]	train-mlogloss:0.787207	eval-mlogloss:0.935956
[159]	train-mlogloss:0.786345	eval-mlogloss:0.935785
[160]	train-mlogloss:0.785283	eval-mlogloss:0.935403
[161]	train-mlogloss:0.784243	eval-mlogloss:0.935135
[162]	train-mlogloss:0.782965	eval-mlogloss:0.934605
[163]	train-mlogloss:0.781806	eval-mlogloss:0.934388
[164]	train-mlogloss:0.78089	eval-mlogloss:0.93425
[165]	train-mlogloss:0.779748	eval-mlogloss:0.933886
[166]	train-mlogloss:0.778825	eval-mlogloss:0.933733
[167]	train-mlogloss:0.77766	eval-mlogloss:0.933265
[168]	train-mlogloss:0.776652	eval-mlogloss:0.933066
[169]	train-mlogloss:0.775609	eval-mlogloss:0.932563
[170]	train-mlogloss:0.774652	eval-mlogloss:0.932405
[171]	train-mlogloss:0.773613	eval-mlogloss:0.93195
[172]	train-mlogloss:0.772634	eval-mlogloss:0.931506
[173]	train-mlogloss:0.771706	eval-mlogloss:0.931493
[174]	train-mlogloss:0.770685	eval-mlogloss:0.931085
[175]	train-mlogloss:0.769758	eval-mlogloss:0.9308

In [78]:
dmtest = xgb.DMatrix(test_array) 
pred_prob_xgb = bst.predict(dmtest)

In [79]:
proba_df = pd.DataFrame(pred_prob_xgb, columns=rf.classes_)
proba_df.columns = proba_df.columns.map(lambda x: "TripType_" + str(x))
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)
sub_df.to_csv("../submission_0409_noProb_xgb.csv", index=False)